In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import random
from joblib import load, dump
import gc
from tqdm import tqdm

random.seed(42)

In [2]:
image_loc = ['HMB_1','HMB_2','HMB_4','HMB_5','HMB_6']

## Load in data

In [4]:
model_inputs = pd.read_csv('./Data/cleaned_data/model_inputs.csv')

In [5]:
model_inputs.columns

Index(['orientation.x', 'orientation.y', 'orientation.z', 'orientation.w',
       'orientation_covariance_0', 'orientation_covariance_1',
       'orientation_covariance_2', 'orientation_covariance_3',
       'orientation_covariance_4', 'orientation_covariance_5',
       'orientation_covariance_6', 'orientation_covariance_7',
       'orientation_covariance_8', 'angular_velocity.x', 'angular_velocity.y',
       'angular_velocity.z', 'angular_velocity_covariance_0',
       'angular_velocity_covariance_1', 'angular_velocity_covariance_2',
       'angular_velocity_covariance_3', 'angular_velocity_covariance_4',
       'angular_velocity_covariance_5', 'angular_velocity_covariance_6',
       'angular_velocity_covariance_7', 'angular_velocity_covariance_8',
       'linear_acceleration.x', 'linear_acceleration.y',
       'linear_acceleration.z', 'linear_acceleration_covariance_0',
       'linear_acceleration_covariance_1', 'linear_acceleration_covariance_2',
       'linear_acceleration_covarian

In [11]:
inputs = ['orientation.x', 'orientation.y', 'orientation.z', 'orientation.w','orientation_covariance_0', 
          'orientation_covariance_1','orientation_covariance_2', 'orientation_covariance_3','orientation_covariance_4', 'orientation_covariance_5','orientation_covariance_6', 'orientation_covariance_7',
          'orientation_covariance_8', 'angular_velocity.x', 'angular_velocity.y','angular_velocity.z', 'angular_velocity_covariance_0',
          'angular_velocity_covariance_1', 'angular_velocity_covariance_2','angular_velocity_covariance_3', 'angular_velocity_covariance_4',
          'angular_velocity_covariance_5', 'angular_velocity_covariance_6','angular_velocity_covariance_7', 'angular_velocity_covariance_8',
          'linear_acceleration.x', 'linear_acceleration.y','linear_acceleration.z', 'linear_acceleration_covariance_0','linear_acceleration_covariance_1', 'linear_acceleration_covariance_2',
          'linear_acceleration_covariance_3', 'linear_acceleration_covariance_4','linear_acceleration_covariance_5', 'linear_acceleration_covariance_6',
          'linear_acceleration_covariance_7', 'linear_acceleration_covariance_8','steering_wheel_angle', 'steering_wheel_torque', 'speed',
          'steering_enabled', 'throttle_pc', 'throttle_rate', 'engine_rpm','brake_torque_actual', 'wheel_torque_actual', 'accel_over_ground',
          'hill_start_assist_system_status', 'hill_start_assist_mode','abs_active', 'abs_enabled', 'stab_active', 'stab_enabled','trac_active', 'trac_enabled', 'parking_brake_enabled', 'stationary',
          'throttle_pedal_input', 'throttle_pedal_cmd', 'throttle_pedal_output','throttle_enabled', 'front_left_wheel', 'front_right_wheel',
          'rear_left_wheel', 'rear_right_wheel', 'brake_pedal_input','brake_pedal_cmd', 'brake_pedal_output', 'torque_input', 'torque_cmd',
          'torque_output', 'brake_lights_input', 'brake_lights_output','brake_enabled']

###  Run functions

In [9]:
def display_random_predictions(model, model_columns, columns_to_predict, X_test, y_test, num_images=5, save_dir='./prediction_images'):
    """
    Display random images from the test set with true and predicted values, and save them to a folder.
    
    Parameters:
    - model: The trained CNN model
    - columns_to_predict: List of column names to predict (e.g., ['speed', 'steering_angle', 'torque'])
    - X_test: Test images
    - y_test: Ground truth values for the columns being predicted
    - num_images: Number of random images to display
    - save_dir: Directory to save the images (default is './prediction_images')
    """
    
    # Ensure the save directory exists
    os.makedirs(save_dir, exist_ok=True)
    
    # Make predictions on the test data
    predictions = model.predict(X_test)
    
    # Ensure y_test is a pandas DataFrame
    if isinstance(y_test, np.ndarray):
        y_test = pd.DataFrame(y_test, columns=model_columns)
    
    # Extract true and predicted values for each column in columns_to_predict
    true_values = {col: y_test[col].values for col in columns_to_predict}
    pred_values = {col: predictions[:, i] for i, col in enumerate(columns_to_predict)}
    
    # Randomly select indices from the test set
    random_indices = np.random.choice(len(X_test), num_images)
    
    # Plot the images along with the true and predicted values
    for idx in random_indices:
        # Get the image (ensure it's a 3-channel image for display)
        image = X_test[idx]
        
        # If the image has more than 3 channels (e.g., 9 channels), we display only the first 3 channels
        if image.shape[-1] > 3:
            image = image[..., :3]  # Take the first 3 channels (e.g., for RGB)
        
        # Get true and predicted values for each column
        true_output = [true_values[col][idx] for col in columns_to_predict]
        pred_output = [pred_values[col][idx] for col in columns_to_predict]
        
        # Create a string for the title
        title = "True vs Predicted\n"
        title += "\n".join([f"{col.replace('_',' ').title()}: {true_output[i]:.2f} vs {pred_output[i]:.2f}" 
                            for i, col in enumerate(columns_to_predict)])
        
        # Plot the image
        plt.figure(figsize=(6, 6))
        plt.imshow(image)
        plt.axis('off')
        plt.title(title)
        
        # Save the image with a unique filename in the provided folder
        img_path = os.path.join(save_dir, f'image_{idx}.png')
        plt.tight_layout()
        plt.savefig(img_path)
        plt.close()  # Close the figure to avoid memory issues
        
        print(f"Saved image to {img_path}")
        plt.show()


In [8]:
def prepare_data_with_labels(df, image_columns, label_columns, target_size=(224, 224)):
    """
    Prepares X (images) and y (labels) for training a CNN.
    Parameters:
        df (pd.DataFrame): DataFrame containing image paths and labels.
        image_columns (list): List of column names with image paths (e.g., ['left_image_path', 'center_image_path', 'right_image_path']).
        label_columns (list): List of column names for labels (e.g., ['steering_angle', 'throttle_cp']).
        target_size (tuple): Target size for resizing images (width, height).
    Returns:
        np.ndarray: Array of image data (X) with shape (n_samples, height, width, channels).
        np.ndarray: Array of labels (y).
    """ 
    X_images = []
    y_labels = []
    
    # Add progress bar
    print("Processing images...")
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        row_images = []
        valid_images = True
        
        # Process images for the current row
        for col in image_columns:
            image_path = row[col]
            
            if os.path.exists(image_path):
                try:
                    # Open and convert to RGB to ensure 3 channels
                    img = Image.open(image_path).convert('RGB')
                    # Resize image
                    img = img.resize(target_size)
                    # Convert to numpy array and normalize
                    img_array = np.array(img, dtype=np.float32) / 255.0
                    
                    # Verify image shape
                    if img_array.shape != (*target_size, 3):
                        print(f"Unexpected image shape for {image_path}: {img_array.shape}")
                        valid_images = False
                        break
                    
                    row_images.append(img_array)
                except Exception as e:
                    print(f"Error processing image {image_path}: {e}")
                    valid_images = False
                    break
            else:
                print(f"Image not found: {image_path}")
                valid_images = False
                break
        
        # Only process row if all images were valid
        if valid_images and len(row_images) == len(image_columns):
            # Concatenate images along the channel dimension
            # Each image is (height, width, 3), so final shape will be (height, width, 3 * num_cameras)
            combined_image = np.concatenate(row_images, axis=-1)
            X_images.append(combined_image)
            y_labels.append(row[label_columns].values)
    
    if not X_images:
        raise ValueError("No valid images were processed")
    
    # Convert lists to numpy arrays
    X = np.array(X_images, dtype=np.float32)
    y = np.array(y_labels, dtype=np.float32)
    
    # Print shapes for verification
    print(f"\nFinal data shapes:")
    print(f"X shape: {X.shape}")  # Should be (n_samples, height, width, 3 * num_cameras)
    print(f"y shape: {y.shape}")  # Should be (n_samples, num_labels)
    
    return X, y

In [9]:
X, y = prepare_data_with_labels(model_inputs,['left_image_path', 'center_image_path', 'right_image_path'],inputs)
# model_inputs.columns[:36][39:]

Processing images...


100%|████████████████████████████████████| 33782/33782 [15:28<00:00, 36.38it/s]



Final data shapes:
X shape: (33782, 224, 224, 9)
y shape: (33782, 74)


### Save the processed images for faster model testing

In [10]:
dump(X, "./Data/saved_variables/X_model_inputs.pkl")
dump(y, "./Data/saved_variables/y_model_inputs.pkl")

['./Data/saved_variables/y_model_inputs.pkl']

In [3]:
X_model_inputs = load("./Data/saved_variables/X_model_inputs.pkl")
y_model_inputs = load("./Data/saved_variables/y_model_inputs.pkl")

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_model_inputs,
                                                    y_model_inputs,
                                                   shuffle = False)

In [5]:
print(X_train.shape)
print(X_test.shape)

(25336, 224, 224, 9)
(8446, 224, 224, 9)


### Run the model

In [15]:
def create_model(input_shape):
    model = Sequential()
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 4)

    # Input layer
    model.add(Input(shape=input_shape))

    # First convolutional block
    model.add(Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Second convolutional block
    model.add(Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Third convolutional block
    model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Fourth convolutional block
    model.add(Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Fifth convolutional block
    model.add(Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Sixth convolutional block
    model.add(Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Seventh convolutional block
    model.add(Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Eighth convolutional block
    model.add(Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'))
    model.add(BatchNormalization())

    # Flatten and fully connected layers
    model.add(Flatten())
    model.add(Dense(units=512, activation='relu'))
    model.add(Dropout(0.5))  
    model.add(Dense(units=256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=256, activation='relu'))
    model.add(Dropout(0.5))
    
    # Output layer
    model.add(Dense(units=74, activation='linear'))
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae','mse'])

    history = model.fit(X_train,y_train,
                       validation_data = (X_test,y_test),
                       epochs = 50,
                        callbacks = early_stopping,
                       batch_size = 128)

    return model, history
    
input_shape = (224,224,9)
model, history = create_model(input_shape)

Epoch 1/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 652s 3s/step - loss: 32008.4980 - mae: 62.3140 - mse: 32008.4980 - val_loss: 32173.8184 - val_mae: 31.1190 - val_mse: 32173.8184
Epoch 2/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 472s 2s/step - loss: 7747.3257 - mae: 24.6518 - mse: 7747.3257 - val_loss: 44660.1992 - val_mae: 34.4120 - val_mse: 44660.1992
Epoch 3/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 484s 2s/step - loss: 6400.5225 - mae: 16.9176 - mse: 6400.5225 - val_loss: 40767.9375 - val_mae: 33.3299 - val_mse: 40767.9375
Epoch 4/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 473s 2s/step - loss: 5771.1865 - mae: 14.7109 - mse: 5771.1865 - val_loss: 45922.7578 - val_mae: 34.5691 - val_mse: 45922.7578
Epoch 5/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 496s 3s/step - loss: 5714.2773 - mae: 14.1866 - mse: 5714.2773 - val_loss: 39914.2148 - val_mae: 32.7313 - val_mse: 39914.2148


In [16]:
model.evaluate(X_test,y_test)

264/264 ━━━━━━━━━━━━━━━━━━━━ 28s 106ms/step - loss: 36235.0117 - mae: 29.6265 - mse: 36235.0117


[39914.2109375, 32.731346130371094, 39914.2109375]

## Save the Model

In [17]:
model.save('./Data/full_models/model_inputs.keras')

In [6]:
model = load_model('./Data/full_models/model_inputs.keras')

In [18]:
model_inputs.columns

Index(['orientation.x', 'orientation.y', 'orientation.z', 'orientation.w',
       'orientation_covariance_0', 'orientation_covariance_1',
       'orientation_covariance_2', 'orientation_covariance_3',
       'orientation_covariance_4', 'orientation_covariance_5',
       'orientation_covariance_6', 'orientation_covariance_7',
       'orientation_covariance_8', 'angular_velocity.x', 'angular_velocity.y',
       'angular_velocity.z', 'angular_velocity_covariance_0',
       'angular_velocity_covariance_1', 'angular_velocity_covariance_2',
       'angular_velocity_covariance_3', 'angular_velocity_covariance_4',
       'angular_velocity_covariance_5', 'angular_velocity_covariance_6',
       'angular_velocity_covariance_7', 'angular_velocity_covariance_8',
       'linear_acceleration.x', 'linear_acceleration.y',
       'linear_acceleration.z', 'linear_acceleration_covariance_0',
       'linear_acceleration_covariance_1', 'linear_acceleration_covariance_2',
       'linear_acceleration_covarian

## Get out random images for review

In [27]:
display_random_predictions(model,inputs,['abs_active','brake_pedal_input','brake_pedal_output','brake_enabled'],X_test,y_test,save_dir = './prediction_images/brake')

264/264 ━━━━━━━━━━━━━━━━━━━━ 30s 115ms/step
Saved image to ./prediction_images/brake\image_1121.png
Saved image to ./prediction_images/brake\image_5598.png
Saved image to ./prediction_images/brake\image_2410.png
Saved image to ./prediction_images/brake\image_1374.png
Saved image to ./prediction_images/brake\image_7307.png


In [12]:
display_random_predictions(model,inputs,['speed','throttle_pedal_output','throttle_pedal_input'],X_test,y_test,save_dir = './prediction_images/speed')

264/264 ━━━━━━━━━━━━━━━━━━━━ 29s 107ms/step
Saved image to ./prediction_images/speed\image_8362.png
Saved image to ./prediction_images/speed\image_4168.png
Saved image to ./prediction_images/speed\image_3813.png
Saved image to ./prediction_images/speed\image_4154.png
Saved image to ./prediction_images/speed\image_3163.png


In [29]:
display_random_predictions(model,inputs,['brake_torque_actual','wheel_torque_actual','accel_over_ground'],X_test,y_test,save_dir = './prediction_images/brake')

264/264 ━━━━━━━━━━━━━━━━━━━━ 30s 115ms/step
Saved image to ./prediction_images/brake\image_4987.png
Saved image to ./prediction_images/brake\image_6354.png
Saved image to ./prediction_images/brake\image_4198.png
Saved image to ./prediction_images/brake\image_4374.png
Saved image to ./prediction_images/brake\image_3928.png


In [31]:
display_random_predictions(model,inputs,['throttle_pedal_input','throttle_pedal_output','engine_rpm'],X_test,y_test,save_dir = './prediction_images/throttle')

264/264 ━━━━━━━━━━━━━━━━━━━━ 31s 116ms/step
Saved image to ./prediction_images/throttle\image_7974.png
Saved image to ./prediction_images/throttle\image_4028.png
Saved image to ./prediction_images/throttle\image_7780.png
Saved image to ./prediction_images/throttle\image_5424.png
Saved image to ./prediction_images/throttle\image_700.png


In [30]:
display_random_predictions(model,inputs,['steering_wheel_angle','steering_enabled','steering_wheel_torque'],X_test,y_test,save_dir = './prediction_images/steer')

264/264 ━━━━━━━━━━━━━━━━━━━━ 30s 115ms/step
Saved image to ./prediction_images/steer\image_3398.png
Saved image to ./prediction_images/steer\image_628.png
Saved image to ./prediction_images/steer\image_3041.png
Saved image to ./prediction_images/steer\image_4697.png
Saved image to ./prediction_images/steer\image_5683.png


### Delete all variables and recollect memory for faster model fitting and so no errors occur where there is no memory left available

In [ ]:
del X, y, X_train, X_test, y_train, y_test, model, history, X_model_inputs, y_model_inputs

In [ ]:
gc.collect()